In [1]:
# import necessary packages
import pandas as pd
import numpy as np

$\beta_{ij}$ are the demand-weighted distances used to determine the center from node $i$ to node $j$.  The formula to calculate $\beta_{ij}$ is given by
$$\beta_{ij} = \frac{h_i h_j d(i,\ j)}{h_i + h_j}$$
where $h_i$ is the demand of node $i$, and $d(i,\ j)$ is the shortest path distance from node $i$ to node $j$.

In [2]:
# Function to calculate Beta_ij
def Beta(A, B, Demand, Distance):
    return (Demand[A] * Demand[B] * Distance.loc[A, B]) / (Demand[A] + Demand[B])

In [3]:
# Initialize Demand Series and Shortest Path Distance Dataframe
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
Demand = pd.Series([8, 6, 23, 50, 40, 13, 25, 12], index = nodes)

shortDist = np.array([[ 0, 71, 52, 15, 35, 28, 57, 72],  # Row A
                      [71,  0, 19, 56, 36, 69, 58, 73],  # Row B
                      [52, 19,  0, 37, 17, 50, 39, 54],  # Row C
                      [15, 56, 37,  0, 20, 13, 42, 57],  # Row D
                      [35, 36, 17, 20,  0, 33, 22, 37],  # Row E
                      [28, 69, 50, 13, 33,  0, 55, 70],  # Row F
                      [57, 58, 39, 42, 22, 55,  0, 15],  # Row G
                      [72, 73, 54, 57, 37, 70, 15,  0]]) # Row H
Distance = pd.DataFrame(shortDist, index = nodes, columns = nodes)

print(Demand)
print(Distance)

A     8
B     6
C    23
D    50
E    40
F    13
G    25
H    12
dtype: int64
    A   B   C   D   E   F   G   H
A   0  71  52  15  35  28  57  72
B  71   0  19  56  36  69  58  73
C  52  19   0  37  17  50  39  54
D  15  56  37   0  20  13  42  57
E  35  36  17  20   0  33  22  37
F  28  69  50  13  33   0  55  70
G  57  58  39  42  22  55   0  15
H  72  73  54  57  37  70  15   0


In [4]:
# Create Beta Matrix
BetaMatrix = pd.DataFrame(0, index = nodes, columns = nodes)
for rowIndex, row in Distance.iterrows():
    for colIndex, val in row.iteritems():
        b_ij = Beta(rowIndex, colIndex, Demand, Distance)
        BetaMatrix.loc[rowIndex, colIndex] = b_ij
        
pd.set_option('precision', 2)  # Set number of decimal places for printing
print(BetaMatrix)

        A       B       C       D       E       F       G       H
A    0.00  243.43  308.65  103.45  233.33  138.67  345.45  345.60
B  243.43    0.00   90.41  300.00  187.83  283.26  280.65  292.00
C  308.65   90.41    0.00  582.88  248.25  415.28  467.19  425.83
D  103.45  300.00  582.88    0.00  444.44  134.13  700.00  551.61
E  233.33  187.83  248.25  444.44    0.00  323.77  338.46  341.54
F  138.67  283.26  415.28  134.13  323.77    0.00  470.39  436.80
G  345.45  280.65  467.19  700.00  338.46  470.39    0.00  121.62
H  345.60  292.00  425.83  551.61  341.54  436.80  121.62    0.00


In [5]:
# Find Maximum Value and Location
print(BetaMatrix.max().max())
colMaxID = BetaMatrix.max().idxmax()
rowMaxID = BetaMatrix.loc[:, str(colMaxID)].idxmax()
print("Row %s      Column %s" %(rowMaxID, colMaxID))
print("Row %s      Column %s" %(colMaxID, rowMaxID))

700.0
Row G      Column D
Row D      Column G


If $S$ and $T$ are the optimal nodes that correspond to $\beta_{ST} = \text{max}_{i,\ j}\ (\beta_{ij})$, then the optimal location is along the shortest path edge from $S$ to $T$.

The location is a distance of $\frac{h_T \, d(S,\ T)}{h_S + h_T}$ units from node S, and
the location is a distance of $\frac{h_S \, d(S,\ T)}{h_S + h_T}$ units from node T.

In [6]:
# Determine Absolute Location (on unique path from rowMaxID to colMaxID)
print( "Distance from %s:  %.2f" %(rowMaxID, 
                                    (Demand[colMaxID] * Distance.loc[rowMaxID, colMaxID]) / 
                                    (Demand[rowMaxID] + Demand[colMaxID])) )
print( "Distance from %s:  %.2f" %(colMaxID,  
                                    (Demand[rowMaxID] * Distance.loc[rowMaxID, colMaxID]) /  
                                    (Demand[rowMaxID] + Demand[colMaxID])) )

Distance from G:  28.00
Distance from D:  14.00


In [7]:
# Figure out what Demand['E'] needs to be to change solution
for name in nodes:
    if name != 'E':
        print(name + "  {:>8.2f}".format(700*Demand[name] / (Demand[name] * Distance.loc[name, 'E'] - 700)))

A    -13.33
B     -8.68
C    -52.10
D    116.67
F    -33.58
G   -116.67
H    -32.81


Since negative demands are not feasible, we would say that the demand at node $E$ would have to be <br>
at least 117 units for the location of the
weighted absolute 1-center to change.